In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [3]:
url = 'https://www.themoviedb.org/movie'

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

source_data = requests.get(url, headers = header).text

In [4]:
soup_data = BeautifulSoup(source_data,'lxml')

In [5]:
movie1 = soup_data.find('div',class_='card style_1')

In [6]:
base_url = "https://www.themoviedb.org/movie/"
exp = str(movie1.find('a'))[30:38]  
digits = re.findall("\d",exp) 
extension = "".join(digits)
movie_url = base_url + extension 

In [7]:
source_movie_data = requests.get(movie_url, headers = header).text

In [8]:
soup_movie_data = BeautifulSoup(source_movie_data,'lxml')

In [9]:
movie_name = soup_movie_data.find('h2').find('a').text
movie_name

'Rebel Moon - Part One: A Child of Fire'

In [10]:
release_date = soup_movie_data.find('span',{'class':'release'}).text.replace('\n','').replace('(IN)','').strip()
print(release_date)

12/15/2023 (US)


In [11]:
exp = '\d{2}.\d{2}.\d{4}'
ans3 = re.findall(exp,release_date)
print(ans3[0])

12/15/2023


In [12]:
rating = int(float(str(soup_movie_data.find('div',{'class':'user_score_chart'}))[69:73]))
rating

65

In [13]:
people = soup_movie_data.find('ol',{'class':'people no_image'}).find_all('li')
people

[<li class="profile">
 <p><a href="/person/15217-zack-snyder">Zack Snyder</a></p>
 <p class="character">Director, Screenplay, Story</p>
 </li>,
 <li class="profile">
 <p><a href="/person/17285-kurt-johnstad">Kurt Johnstad</a></p>
 <p class="character">Screenplay</p>
 </li>,
 <li class="profile">
 <p><a href="/person/2104243-shay-hatten">Shay Hatten</a></p>
 <p class="character">Screenplay</p>
 </li>]

In [14]:
for person in people:
    person_str = str(person)
    if 'Director' in person_str:
        director = person.find('a').text
print(director)

Zack Snyder


In [15]:
runtime = soup_movie_data.find('span',class_='runtime').text.replace('\n','').strip()
runtime

'2h 14m'

In [16]:
genre = soup_movie_data.find('span',class_='genres').text.replace('\n','').replace('\xa0','')
genre

'Science Fiction'

In [17]:
import requests
from bs4 import BeautifulSoup

In [18]:
url = 'https://www.themoviedb.org/movie' 

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'}

source_data = requests.get(url, headers = header).text
soup_data = BeautifulSoup(source_data, 'lxml')

In [19]:
all_url = []
for i in range(1, 51):
    page = f"?page={i}"
    all_url.append(url + page)
all_url

['https://www.themoviedb.org/movie?page=1',
 'https://www.themoviedb.org/movie?page=2',
 'https://www.themoviedb.org/movie?page=3',
 'https://www.themoviedb.org/movie?page=4',
 'https://www.themoviedb.org/movie?page=5',
 'https://www.themoviedb.org/movie?page=6',
 'https://www.themoviedb.org/movie?page=7',
 'https://www.themoviedb.org/movie?page=8',
 'https://www.themoviedb.org/movie?page=9',
 'https://www.themoviedb.org/movie?page=10',
 'https://www.themoviedb.org/movie?page=11',
 'https://www.themoviedb.org/movie?page=12',
 'https://www.themoviedb.org/movie?page=13',
 'https://www.themoviedb.org/movie?page=14',
 'https://www.themoviedb.org/movie?page=15',
 'https://www.themoviedb.org/movie?page=16',
 'https://www.themoviedb.org/movie?page=17',
 'https://www.themoviedb.org/movie?page=18',
 'https://www.themoviedb.org/movie?page=19',
 'https://www.themoviedb.org/movie?page=20',
 'https://www.themoviedb.org/movie?page=21',
 'https://www.themoviedb.org/movie?page=22',
 'https://www.themo

In [20]:
final_data = []

for i in all_url: 
    source_data_all = requests.get(i, headers = header).text
    soup_data_all = BeautifulSoup(source_data_all, 'lxml')
    all_card = soup_data_all.find_all('div', class_='card style_1')
    
    for item in all_card:
        link = str(item.find('a'))[29:37]
        new_url = url + link
        movie = requests.get(new_url,headers = header).text
        movie_soup = BeautifulSoup(movie, 'lxml')
        
        # For finding Name, Date, Rating, Director, Duration, Genre
        movie_head = movie_soup.find('section', id='original_header')
        
        # Movie Name
        try:
            movie_name = movie_head.find('h2').find('a').text
        except:
            movie_name = None
        
        # Release date of a movie
        try:
            release_date = movie_head.find('span', class_='release').text.replace('\n','').strip().split(" ")[0]
        except:
            release_date = None
        
        # We fetched the attribute value from div tag for rating as its not in text
        try:
            rating = movie_head.find('div', class_='user_score_chart')["data-percent"]
        except:
            rating = None
        
        # As some movies has multiple directors, loop is used to 
        try:
            production = movie_head.find('ol', class_='people no_image')
            profile = production.find_all('li', class_='profile')
            director = ""
            for i in range(0, len(profile)):
                character = str(profile[i].find_all('p')[1].text)
                if 'Director' in character:
                    director+= f"{profile[i].find('a').text},"
            director = director.rstrip(',')
        except:
            director = None
        
        # dur has none value in pages where it doesnt exist hence has no attribute text
        try:
            duration = movie_head.find('span', class_='runtime').text.replace('\n','').strip()
        except:
            duration = None
        # Same thing using if statement
#         dur = movie_head.find('span', class_='runtime').text.replace('\n','').strip()
#         if(dur is None):
#             duration = None
#         else:
#             duration = dur.text.replace('\n','').strip()
        
        # Genre of a movie
        try:
            genre = movie_head.find('span', class_='genres').text.replace('\n','').replace('\xa0','')
        except:
            genre = None
        
        # All details stored in the dictionary
        movie_dict = {
            'Movie Name' : movie_name,
            'Release Date' : release_date,
            'Rating' : rating,
            'Director' : director,
            'Duration' : duration,
            'Genre' : genre
        }
        
        # Finally each movie dictionary is appended to the list
        final_data.append(movie_dict)
final_data

[{'Movie Name': 'Rebel Moon - Part One: A Child of Fire',
  'Release Date': '12/15/2023',
  'Rating': '65.0',
  'Director': 'Zack Snyder',
  'Duration': '2h 14m',
  'Genre': 'Science Fiction'},
 {'Movie Name': 'Aquaman and the Lost Kingdom',
  'Release Date': '12/22/2023',
  'Rating': '66.0',
  'Director': 'James Wan',
  'Duration': '2h 4m',
  'Genre': 'Action,Adventure,Fantasy'},
 {'Movie Name': 'The Hunger Games: The Ballad of Songbirds & Snakes',
  'Release Date': '11/17/2023',
  'Rating': '73.0',
  'Director': 'Francis Lawrence',
  'Duration': '2h 37m',
  'Genre': 'Drama,Science Fiction,Action'},
 {'Movie Name': 'The Family Plan',
  'Release Date': '12/15/2023',
  'Rating': '74.0',
  'Director': 'Simon Cellan Jones',
  'Duration': '1h 59m',
  'Genre': 'Action,Comedy'},
 {'Movie Name': 'Wonka',
  'Release Date': '12/08/2023',
  'Rating': '71.0',
  'Director': 'Paul King',
  'Duration': '1h 57m',
  'Genre': 'Comedy,Family,Fantasy'},
 {'Movie Name': 'Silent Night',
  'Release Date': '

In [21]:
import pandas as pd

In [22]:
pf = pd.DataFrame(final_data)
pf

,Movie Name,Release Date,Rating,Director,Duration,Genre
0,Rebel Moon - Part One: A Child of Fire,12/15/2023,65.0,Zack Snyder,2h 14m,Science Fiction
1,Aquaman and the Lost Kingdom,12/22/2023,66.0,James Wan,2h 4m,"Action,Adventure,Fantasy"
2,The Hunger Games: The Ballad of Songbirds & Sn...,11/17/2023,73.0,Francis Lawrence,2h 37m,"Drama,Science Fiction,Action"
3,The Family Plan,12/15/2023,74.0,Simon Cellan Jones,1h 59m,"Action,Comedy"
4,Wonka,12/08/2023,71.0,Paul King,1h 57m,"Comedy,Family,Fantasy"
...,...,...,...,...,...,...
995,Bridge to Terabithia,02/16/2007,73.0,Gábor Csupó,1h 36m,"Adventure,Drama,Family"
996,Apocalypto,12/08/2006,76.0,Mel Gibson,2h 19m,"Action,Drama,History"
997,Ip Man: The Awakening,09/16/2021,69.0,"Zhang Zhulin,Li Xijie",1h 16m,"Action,Adventure,History,Drama"
998,Air,04/05/2023,74.0,Ben Affleck,1h 51m,"Drama,History"


In [23]:
pf.to_excel('themoviedb.xlsx')